<a href="https://colab.research.google.com/github/zain711/dcipher/blob/main/BrazilNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic
!pip install bertopic[visualization]
!pip install bert-extractive-summarizer
!pip install plotly
!pip install torch
!pip install tensorflow
!pip install flair
!pip install googletrans==3.1.0a0
!pip install gensim==4.0.0b

In [ ]:
!pip install 'umap-learn==0.5.0'

In [ ]:
!git clone https://github.com/zain711/BERTopic.git

fatal: destination path 'BERTopic' already exists and is not an empty directory.


In [ ]:
!cp /content/BERTopic/bertopic/_bertopic.py /usr/local/lib/python3.7/dist-packages/bertopic/_bertopic.py

In [ ]:
import json
import pandas as pd
import torch
from bertopic import BERTopic
import os
import nltk 
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer , WordNetLemmatizer
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import six
from google.cloud import translate_v2 as translate
import getpass

APIKEY = getpass.getpass()

··········


In [ ]:
!pip list -v | grep bertopic

bertopic                      0.7.0          /usr/local/lib/python3.7/dist-packages pip      


# Preprocessing Text

In [ ]:
#previous vaccine forums
data = pd.read_table('/content/drive/MyDrive/BrazilAnxNews.tsv')
docs_raw = list(data.loc[:, "matches.value"].values)
times = list(data.loc[:, "Date"].values)

In [ ]:
data.head(5)

,matches.value,Date
0,Mais negociações de dívidas e de relações de t...,"""2020-10"""
1,"Verde : para harmonia, equilíbrio e serenidad...","""2021-03"""
2,"Quando as chamas se tornam mais fortes, os pol...","""2019-10"""
3,As pessoas precisam do contato afetivo com out...,"""2020-06"""
4,"Mas vamos esperar um pouco para salgá-los, cas...","""2020-08"""


In [ ]:
import re
wnl = WordNetLemmatizer()
ps = PorterStemmer()

#tokens = list(data.loc[:, "tokens"].values)
docs = []
for_embed = []
rem_num = []
my_tokens = []

docs_lower = [k.lower() for k in docs_raw]
#remove numbers and symbols from token list and lemmatize
for b in docs_lower:
  rem_num.append(re.sub('[0-9]+', '', b)) #remove numerical tokens

for a in rem_num:
  docs.append(re.sub(r'[^\w]', ' ', a))

#for c in new_tokens:
  #my_tokens.append(" ".join([wnl.lemmatize(a) for a in c.split()]))

#split tokens for word embeddings
for c in docs:
  for_embed.append(c.split())

In [ ]:
#clean times for topic evolution visualization
clean_times = []
for a in times:
  #ts = pd.to_datetime(str(a)) 
  #d = a.strftime('%Y.%m')
  clean_times.append(a)
clean_times = [i.replace('"', '') for i in times]

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
import matplotlib.pyplot as plt 

#calculate and graph loss of embeddings
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''
    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0
        self.epochs = []
        self.losses = []

    def on_epoch_end(self, modeling):
        plt.axis([0, 100, 0, 3000000])
        loss = modeling.running_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        epochs = self.epoch
        losses = loss_now
        plt.plot(self.epoch, loss_now, '.r-')
        self.epoch += 1
        if (self.epoch == 100):
           plt.show()

# **FINE TUNED FASTTEXT MODEL**

In [ ]:
!pip install gensim==4.0.0b

In [ ]:
from gensim.models.fasttext import FastText, load_facebook_model, save_facebook_model
from multiprocessing import cpu_count
from typing import List
from math import log10
import numpy as np
from gensim.test.utils import datapath

model_path = "/content/drive/MyDrive/portuguese.bin"
model = load_facebook_model(datapath(model_path))
model.workers = cpu_count() - 2

In [ ]:
model.build_vocab(corpus_iterable=for_embed, update=False, keep_raw_vocab=False)

In [ ]:
num_docs = len(for_embed)
num_tokens = sum([len(doc) for doc in for_embed])
num_epochs = min(round(100.0 / (log10(num_docs + 1) + 1e-5)), 100)

In [ ]:
model.train(corpus_iterable=for_embed,
            total_examples=num_docs,
            total_words = num_tokens,
            epochs=100)

In [ ]:
# 10 topics, 100 epochs, fine tuned - 0.5557887476532941, 0.45324172191545986
# 10 topics, 100 epochs, with updated embeddings - ~0.44, ~0.38
# 20 epochs, 50 epochs, fine tuned - ~0.45, ~.51

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
import matplotlib.pyplot as plt 

#calculate and graph loss of embeddings
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''
    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0
        self.epochs = []
        self.losses = []

    def on_epoch_end(self, modeling):
        plt.axis([0, 100, 0, 3000000])
        loss = model.get_latest_training_loss()
        #loss_now = loss - self.loss_to_be_subed
        #self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        epochs = self.epoch
        #losses = loss_now
        plt.plot(self.epoch, loss, '.r-')
        self.epoch += 1
        if (self.epoch == 100):
           plt.show()

In [ ]:
# save model
save_facebook_model(model, "finetuned_model2.bin")

In [ ]:
# After training the model
# 1. Word vector extraction
def get_word_vector(word):
    return model.wv.get_vector(word)

# 2. Sentence vector extraction
def get_sentence_vector(text: List[str]):
    def div_norm(x):
        norm_value = np.linalg.norm(x)
        if norm_value > 0:
            return x * (1.0 / norm_value)
        else:
            return x
    
    token_vectors = np.array([div_norm(get_word_vector(token))
                                  for token in text if len(token) > 0])

    if len(token_vectors) == 0:
        return np.zeros(100)

    return np.mean(token_vectors, axis=0)

In [ ]:
word_vectors = model.wv
word_vectors.save("/content/drive/My Drive/fasttext_finetune_port")

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

glove_embedding = WordEmbeddings("/content/drive/My Drive/fasttext_finetune_port")
document_glove_embeddings = DocumentPoolEmbeddings([glove_embedding])

In [ ]:
topic_model = BERTopic(nr_topics=10, embedding_model=document_glove_embeddings)

In [ ]:
topics, prob = topic_model.fit_transform(docs)

In [ ]:
#remove stop words and change n-gram range
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = ['ter', 'ser','tempo','dia','porque','bem','ainda','tudo','pode','estar']
stopped = stopwords.words("portuguese")
for a in stop:
  stopped.append(a)
cv = CountVectorizer(ngram_range=(1, 1), stop_words=stopped)
topic_model.update_topics(docs, topics, vectorizer_model=cv)

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
0,-1,108052
1,596,4316
2,1332,3948
3,1303,2872
4,1471,2266
5,1272,2232
6,1021,1104
7,1170,929
8,1355,887
9,425,740


In [ ]:
topic_model.get_topics()

{-1: [('sozinho', 0.01932900669586986),
  ('vida', 0.009538665636648784),
  ('casa', 0.008890337060252173),
  ('segundo', 0.008544458802435248),
  ('feira', 0.007426006293407471),
  ('momento', 0.0072805368176220514),
  ('trabalho', 0.006887420129189832),
  ('todos', 0.006844688144691215),
  ('social', 0.006563051353258985),
  ('conta', 0.006421411084961419)],
 425: [('avião', 0.08593079122704328),
  ('drone', 0.05698965621447832),
  ('feira', 0.03255234835296558),
  ('voo', 0.02681951479232764),
  ('americano', 0.024656412472109396),
  ('estados', 0.022693505223983723),
  ('aéreo', 0.02065457467506787),
  ('bordo', 0.019939037769067476),
  ('país', 0.019525558764249683),
  ('aeronave', 0.018146309511640736)],
 596: [('gol', 0.07834555670027377),
  ('sozinho', 0.05638060504485814),
  ('bola', 0.046907486329332684),
  ('placar', 0.033193504644118556),
  ('jogo', 0.031581262751678274),
  ('partida', 0.02357583983201638),
  ('escanteio', 0.021289294131881364),
  ('campo', 0.01752884157251

In [ ]:
import six
from google.cloud import translate_v2 as translate
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/My Drive/rugged-alloy-311800.json"

def translate_docs(words, lang):
  translate_client = translate.Client()
  result = translate_client.translate(words, target_language='en', source_language=lang)
  return result.get('translatedText')

In [ ]:
topic_list = get_topic_list(topic_model)
translated_list = []
for a in topic_list:
  translated_list.append([translate_docs(i,'pt') for i in a])

In [ ]:
translated_list

[['plane',
  'drone',
  'fair',
  'flight',
  'American',
  'States',
  'air',
  'board',
  'parents',
  'aircraft'],
 ['goal',
  'by myself',
  'ball',
  'scoreboard',
  'game',
  'match',
  'corner kick',
  'field',
  'a tie',
  'crossing'],
 ['health',
  'mental',
  'depression',
  'Social',
  'work',
  'life',
  'stress',
  'professionals',
  'all',
  'teenagers'],
 ['depression',
  'stress',
  'health',
  'treatment',
  'life',
  'alcohol',
  'risk',
  'humor',
  'body',
  'Social'],
 ['health',
  'mental',
  'services',
  'professionals',
  'Social',
  'depression',
  'others',
  'countries',
  'social',
  'others'],
 ['health',
  'stress',
  'by myself',
  'life',
  'fear',
  'help',
  'form',
  'time',
  'Social',
  'important'],
 ['health',
  'mental',
  'doctor',
  'Social',
  'depression',
  'anxiety',
  'life',
  'education',
  'hospital',
  'service'],
 ['by myself',
  'second',
  'House',
  'after',
  'people',
  'case',
  'against',
  'then',
  'crime',
  'time'],
 ['anx

In [ ]:
topic_model.save('/content/drive/My Drive/port_ft_finettune1')
np.save('/content/drive/My Drive/topic_portfine1', topics)

In [ ]:
topic_model = BERTopic.load('/content/drive/My Drive/port_ft_finettune1')
topics = np.load('/content/drive/My Drive/topic_portfine1.npy')

In [ ]:
topics_over_time = topic_model.topics_over_time(docs, topics, clean_times, global_tuning=False, translate = "pt")

In [ ]:
topics_over_time

,Topic,Words,Frequency,Timestamp
0,-1,front wing rotating run place,20,2019-03-01
1,596,"spreading, match, rounds, goal, salah",3,2019-03-01
2,-1,"alone, years, health, mental, fair",2516,2019-04-01
3,425,"airplane, united, deliberate, airbus, states",8,2019-04-01
4,596,"alone, ball, background, step, first",176,2019-04-01
...,...,...,...,...
261,1272,"emergency, aid, value, beneficiary, anxiety",164,2021-03-01
262,1303,"health, mental, professional, public, magic",166,2021-03-01
263,1332,"alone, little friend, then go",234,2021-03-01
264,1355,"anxiety, stress, soothing, depression, insomnia",61,2021-03-01


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n=10)

In [ ]:
topics_over_time['Words'] = topics_over_time['Words'].apply(translate_docs, lang = translate)

In [ ]:
topics_over_time = topic_model.topics_over_time(docs, topics, clean_times, global_tuning=False, translate='pt')

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n=10, translate='pt')

# **Bert**

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer

input_ids = []
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
for a in docs:
  input_ids.append(tokenizer.encode(a, return_tensors='pt'))

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
encoded = []
with torch.no_grad():
  for b in range(len(input_ids)):
    outs = model(input_ids[b])
    encoded.append(outs[0][0, 1:-1])

In [ ]:
import tensorflow as tf
embeddings = tf.make_ndarray(encoded)

In [ ]:
for c in encoded:
  embeddings.append(encoded.numpy())

In [ ]:
np.save('/content/drive/MyDrive/portuguese-bert', embeddings)

In [ ]:
topic_model = BERTopic(nr_topics = 10)

In [ ]:
topics, prob = topic_model.fit_transform(docs, embeddings=embeddings)

In [ ]:
#remove stop words and change n-gram range
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = ['ter', 'ser','tempo','dia','porque','bem','ainda','tudo']
stoped = stopwords.words("portuguese")
for a in stop:
  stoped.append(a)
cv = CountVectorizer(ngram_range=(1, 1), stop_words=stoped)
topic_model.update_topics(docs, topics, vectorizer_model=cv)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
topic_model.get_topics()

# **COHERENCE**

In [ ]:
def get_topic_list(topic_model):
  dictionary = list(topic_model.get_topics().values())
  diction = dictionary[1:] #remove outlier topic
  p = 0
  final_t = []
  for i in range(len(diction)):
    final_t.append([])

  for key in diction:
     for i in key:
          final_t[p].append(i[0])
     p+=1
  return final_t

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
dictionary = corpora.Dictionary(for_embed)
corpus = [dictionary.doc2bow(text) for text in for_embed]

In [ ]:
topics_over_time = topics_over_time[topics_over_time.Topic != -1]

In [ ]:
y = topics_over_time.groupby(["Topic",'Timestamp']).agg({"Words": ', '.join})
local = y.values.tolist()

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
y

Words
Topic Timestamp                                                    
425   2019-04-01       airplane, united, deliberate, airbus, states
      2019-05-01            book, monster, movie, shooting, beijing
      2019-06-01                drone, aerial, american, fair, farm
      2019-07-01                 drone, narrow, ship, flight, boxer
      2019-08-01               pigs, tankers, swine, drone, african
      2019-09-01                    border, drone, base, dead, site
      2019-10-01              army, lord, patiachi, yesica, soldier
      2019-11-01      china, drone, protesters, commercial, chinese
      2019-12-01           china, canteira, aeronautics, saw, black
      2020-01-01               airplane, iranian, board, fair, iraq
      2020-02-01       airplane, Syrian, damaged, black, helicopter
      2020-03-01          airplanes, Syria, Syrians, Syrians, plane
      2020-04-01            book, hero, global, children, countries
      2020-05-01      slaughter, typhoid, public, haunted, mustards
      2020-06-01        Salvador, storm, tropical, shot down, plane
      2020-07-01         boeing, flight, missile, plane, passengers
      2020-08-01     lebanon, israeli, plane, indemnity, phone call
      2020-09-01    fighter, nagorno, diesel, aircraft, separatists
      2020-10-01  democrat, republican, backward, crosse, energetic
      2020-11-01      helicopter, wild, ammunition, country, animal
      2020-12-01      airplane, agency, finished, acoustic, cabinet
      2021-01-01         united, states, democracy, global, defense
      2021-02-01         answers, ambiguities, plane, detailed, lie
      2021-03-01         country, judicial, squid, world, mongolian
596   2019-03-01              spreading, match, rounds, goal, salah
      2019-04-01               alone, ball, background, step, first
      2019-05-01                     goal, alone, ball, game, match
      2019-06-01                     goal, ball, alone, game, score
      2019-07-01                     goal, alone, score, game, ball
      2019-08-01                     alone, ball, game, match, play
      2019-09-01                    goal, alone, game, score, match
      2019-10-01                    alone, score, ball, game, match
      2019-11-01                    alone, game, after, made, match
      2019-12-01                     alone, draw, pass, game, right
      2020-01-01               alone, ball, game, field, scoreboard
      2020-02-01                  goal, alone, ball, score, striker
      2020-03-01                   alone, ball, game, team, defense
      2020-04-01          game, player, alone, club, steering wheel
      2020-05-01            goal, classic, alone, low, championship
      2020-06-01                 goal, alone, score, football, pass
      2020-07-01                goal, alone, game, ball, scoreboard
      2020-08-01                  alone, game, throw, defense, step
      2020-09-01                  alone, score, ball, match, corner
      2020-10-01                   alone, score, game, first, right
      2020-11-01               alone, ball, game, shirt, scoreboard
      2020-12-01                    alone, ball, game, match, score
      2021-01-01                     goal, alone, ball, score, game
      2021-02-01             alone, score, championship, game, draw
      2021-03-01                     alone, ball, score, play, step
1021  2019-04-01         agents, health, depression, mental, edible
      2019-05-01  referendum, fungi, professionals, colorado, fr...
      2019-06-01       illness, depression, health, parents, stress
      2019-07-01    depressed, health, depression, mental, contests
      2019-08-01     health, mental, veterinary, depression, stress
      2019-09-01        health, depression, mental, life, disorders
      2019-10-01       health, depression, life, mental, legitimate
      2019-11-01       thousand, young, domestication, cats, mental
      2019-12-01              women, poverty, elit

In [ ]:
local = local[1:] #remove outlier topic

In [ ]:
b = []
p = 0
topics_local = []
for a in local:
  b.append(a[0].split())

for i in range(len(local)):
   topics_local.append([])

for c in b:
  for e in c:
    topics_local[p].append(e.replace(',', ''))
  p+=1

In [ ]:
cm = CoherenceModel(topics=get_topic_list(topic_model), texts = for_embed, dictionary = dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence
# 10 fine tuned - 0.5557887476532941, 0.45324172191545986
# with updated embeddings ~0.437, ~0.376

# **EXTRA**

In [ ]:
#import six
#from google.cloud import translate_v2 as translate
#import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/My Drive/rugged-alloy-311800.json"


# def translate_names(self,
#                      words,
#    translate_client = translate.Client() 
#    trans_names = words.replace('_', ' ')
#    result = translate_client.translate(trans_names, target_language='en', source_language=lang)

##   return result.get('translatedText')       
       
# if translate:
#   topics_over_time['Name'] = topics_over_time['Name'].apply(self.translate_names, lang = translate)
#   data = topics_over_time.loc[topics_over_time.Topic.isin(selected_topics), :]